In [7]:
import requests
from bs4 import BeautifulSoup
import re
import time

# URL of the Wikipedia page with highest-grossing films
url = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Find the first wikitable on the page (this is assumed to be the main table)
table = soup.find("table", class_="wikitable")

films = []

if table:
    rows = table.find_all("tr")
    # Get header texts to determine column positions.
    headers = [th.get_text(strip=True) for th in rows[0].find_all("th")]
    # Print headers for debugging purposes (optional)
    print("Table headers:", headers)

    # Find index positions for desired columns.
    # Note: The actual header names may vary slightly so adjust accordingly.
    try:
        title_index = [i for i, h in enumerate(headers) if "Title" in h][0]
    except IndexError:
        title_index = None
    try:
        year_index = [i for i, h in enumerate(headers) if re.search(r"Year", h, re.IGNORECASE)][0]
    except IndexError:
        year_index = None
    try:
        gross_index = [i for i, h in enumerate(headers) if re.search(r"gross", h, re.IGNORECASE)][0]
    except IndexError:
        gross_index = None

    # Loop over table rows (skip header row)
    for row in rows[1:]:
        cells = row.find_all(["td", "th"])
        # Skip rows that do not contain enough columns
        if not cells or len(cells) < 3:
            continue

        # Extract film title from the appropriate cell.
        title_cell = cells[title_index] if title_index is not None else None
        title = title_cell.get_text(strip=True) if title_cell else None

        # Extract release year; use a regex to find a 4-digit number.
        year_cell = cells[year_index] if year_index is not None else None
        year_text = year_cell.get_text(strip=True) if year_cell else ""
        try:
            year = int(re.search(r"\d{4}", year_text).group())
        except (AttributeError, ValueError):
            year = None

        # Extract box office gross (as a string, keeping currency symbols)
        gross_cell = cells[gross_index] if gross_index is not None else None
        gross = gross_cell.get_text(strip=True) if gross_cell else None

        # Initialize director and country as None.
        director = None
        country = None

        # If the title cell contains a link, follow it to extract additional details.
        film_link_tag = title_cell.find("a") if title_cell else None
        if film_link_tag and film_link_tag.has_attr("href"):
            film_url = "https://en.wikipedia.org" + film_link_tag["href"]
            film_response = requests.get(film_url)
            film_soup = BeautifulSoup(film_response.text, "html.parser")

            # Look for the infobox that usually contains film details.
            infobox = film_soup.find("table", class_="infobox vevent")
            if infobox:
                for row_infobox in infobox.find_all("tr"):
                    header = row_infobox.find("th")
                    if header:
                        header_text = header.get_text(strip=True)
                        # Extract Director(s)
                        if header_text == "Directed by":
                            director = row_infobox.find("td").get_text(separator=", ", strip=True)
                        # Extract Country of Origin
                        elif header_text == "Country":
                            country = row_infobox.find("td").get_text(separator=", ", strip=True)
            # Pause briefly to be polite with Wikipedia’s servers.
            time.sleep(1)

        films.append({
            "title": title,
            "release_year": year,
            "director": director,
            "box_office": gross,
            "country": country
        })

print(f"Extracted data for {len(films)} films.\n")

Table headers: ['Rank', 'Peak', 'Title', 'Worldwide gross', 'Year', 'Ref']
Extracted data for 50 films.



In [28]:
for film in films:
    print(film)

{'title': 'Avengers: Endgame', 'release_year': 2019, 'director': 'Anthony Russo, Joe Russo', 'box_office': '$2,797,501,328', 'country': 'United States'}
{'title': 'Avatar: The Way of Water', 'release_year': 2022, 'director': 'James Cameron', 'box_office': '$2,320,250,281', 'country': 'United States'}
{'title': 'Titanic', 'release_year': 1997, 'director': 'James Cameron', 'box_office': '$2,257,844,554', 'country': 'United States'}
{'title': 'Star Wars: The Force Awakens', 'release_year': 2015, 'director': 'J. J. Abrams', 'box_office': '$2,068,223,624', 'country': 'United States'}
{'title': 'Avengers: Infinity War', 'release_year': 2018, 'director': 'Anthony Russo, Joe Russo', 'box_office': '$2,048,359,754', 'country': 'United States'}
{'title': 'Ne Zha 2†', 'release_year': 2025, 'director': 'Jiaozi', 'box_office': '$1,985,082,560', 'country': 'China'}
{'title': 'Spider-Man: No Way Home', 'release_year': 2021, 'director': 'Jon Watts', 'box_office': '$1,922,598,800', 'country': 'United St

In [51]:
import re

def clean_box_office(value):
    """
    Removes any characters before the first occurrence of '$'
    and returns the cleaned value.
    """
    if value:
        pos = value.find('$')
        if pos != -1:
            return value[pos:]
    return value

def clean_director(value):
    """
    Removes the unwanted pattern ", [, 1, ]" from the director string.
    """
    if value:
        # Remove every occurrence of the pattern
        return value.replace(", [, 1, ]", "")
    return value

# Clean the films data: update the box_office and director fields,
# and filter out any film with a None value.
clean_films = []
for film in films:
    # Clean the box_office field
    film["box_office"] = clean_box_office(film["box_office"])

    # Clean the director field
    film["director"] = clean_director(film["director"])

    # Keep only entries that have no None values
    if all(v is not None for v in film.values()):
        clean_films.append(film)

print("Cleaned Films Data:")
for film in clean_films:
    print(film)

Cleaned Films Data:
{'title': 'Avengers: Endgame', 'release_year': 2019, 'director': 'Anthony Russo, Joe Russo', 'box_office': '$2,797,501,328', 'country': 'United States'}
{'title': 'Avatar: The Way of Water', 'release_year': 2022, 'director': 'James Cameron', 'box_office': '$2,320,250,281', 'country': 'United States'}
{'title': 'Titanic', 'release_year': 1997, 'director': 'James Cameron', 'box_office': '$2,257,844,554', 'country': 'United States'}
{'title': 'Star Wars: The Force Awakens', 'release_year': 2015, 'director': 'J. J. Abrams', 'box_office': '$2,068,223,624', 'country': 'United States'}
{'title': 'Avengers: Infinity War', 'release_year': 2018, 'director': 'Anthony Russo, Joe Russo', 'box_office': '$2,048,359,754', 'country': 'United States'}
{'title': 'Ne Zha 2†', 'release_year': 2025, 'director': 'Jiaozi', 'box_office': '$1,985,082,560', 'country': 'China'}
{'title': 'Spider-Man: No Way Home', 'release_year': 2021, 'director': 'Jon Watts', 'box_office': '$1,922,598,800', '

In [52]:
len(clean_films)

41

In [53]:
import sqlite3

# Connect to (or create) the SQLite database file
conn = sqlite3.connect('films.db')
cursor = conn.cursor()

# Create the 'films' table with the specified schema
cursor.execute('''
    CREATE TABLE IF NOT EXISTS films (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        release_year INTEGER,
        director TEXT,
        box_office TEXT,
        country TEXT
    )
''')
conn.commit()

# Assuming 'films' is the list of dictionaries you extracted earlier,
# loop over each film entry and insert it into the database.
for film in clean_films:
    cursor.execute('''
        INSERT INTO films (title, release_year, director, box_office, country)
        VALUES (?, ?, ?, ?, ?)
    ''', (
        film.get("title"),
        film.get("release_year"),
        film.get("director"),
        film.get("box_office"),
        film.get("country")
    ))

conn.commit()
conn.close()

print("Data has been successfully inserted into the SQLite database (films.db).")


Data has been successfully inserted into the SQLite database (films.db).


In [54]:
import sqlite3

# Connect to the SQLite database (films.db)
conn = sqlite3.connect('films.db')
cursor = conn.cursor()

# 1. Retrieve All Films
print("Query 1: Retrieve All Films")
cursor.execute("SELECT * FROM films;")
all_films = cursor.fetchall()
for film in all_films:
    print(film)
print("\n" + "-"*50 + "\n")

# 2. Count the Number of Films
print("Query 2: Count the Number of Films")
cursor.execute("SELECT COUNT(*) AS total_films FROM films;")
total_films = cursor.fetchone()[0]
print("Total Films:", total_films)
print("\n" + "-"*50 + "\n")

# 3. List Films Released After 2000
print("Query 3: List Films Released After 2000")
cursor.execute("""
    SELECT title, release_year
    FROM films
    WHERE release_year > 2000
    ORDER BY release_year ASC;
""")
films_after_2000 = cursor.fetchall()
for film in films_after_2000:
    print(film)
print("\n" + "-"*50 + "\n")

# 4. List Unique Countries of Origin
print("Query 4: List Unique Countries of Origin")
cursor.execute("SELECT DISTINCT country FROM films;")
countries = cursor.fetchall()
for country in countries:
    print(country[0])
print("\n" + "-"*50 + "\n")

# 5. Group Films by Director
print("Query 5: Group Films by Director")
cursor.execute("""
    SELECT director, COUNT(*) AS film_count
    FROM films
    GROUP BY director
    ORDER BY film_count DESC;
""")
directors = cursor.fetchall()
for director in directors:
    print(f"Director: {director[0]}, Film Count: {director[1]}")
print("\n" + "-"*50 + "\n")

# 6. Identify Records with Missing Data
print("Query 6: Identify Records with Missing Data")
cursor.execute("""
    SELECT * FROM films
    WHERE director IS NULL OR country IS NULL;
""")
missing_data = cursor.fetchall()
if missing_data:
    for film in missing_data:
        print(film)
else:
    print("No records with missing director or country information found.")
print("\n" + "-"*50 + "\n")

# Close the connection
conn.close()
print("Database queries executed successfully.")


Query 1: Retrieve All Films
(1, 'Avengers: Endgame', 2019, 'Anthony Russo, Joe Russo', '$2,797,501,328', 'United States')
(2, 'Avatar: The Way of Water', 2022, 'James Cameron', '$2,320,250,281', 'United States')
(3, 'Titanic', 1997, 'James Cameron', '$2,257,844,554', 'United States')
(4, 'Star Wars: The Force Awakens', 2015, 'J. J. Abrams', '$2,068,223,624', 'United States')
(5, 'Avengers: Infinity War', 2018, 'Anthony Russo, Joe Russo', '$2,048,359,754', 'United States')
(6, 'Ne Zha 2†', 2025, 'Jiaozi', '$1,985,082,560', 'China')
(7, 'Spider-Man: No Way Home', 2021, 'Jon Watts', '$1,922,598,800', 'United States')
(8, 'Inside Out 2', 2024, 'Kelsey Mann', '$1,698,863,816', 'United States')
(9, 'Jurassic World', 2015, 'Colin Trevorrow', '$1,671,537,444', 'United States')
(10, 'The Lion King', 2019, 'Jon Favreau', '$1,656,943,394', 'United States')
(11, 'The Avengers', 2012, 'Joss Whedon', '$1,518,815,515', 'United States')
(12, 'Top Gun: Maverick', 2022, 'Joseph Kosinski', '$1,495,696,29

In [56]:
import sqlite3
import json

# Connect to the SQLite database
conn = sqlite3.connect('films.db')
cursor = conn.cursor()

# Execute a query to retrieve all rows from the films table
cursor.execute("SELECT * FROM films")
rows = cursor.fetchall()

# Get column names from the cursor description
column_names = [description[0] for description in cursor.description]

# Convert each row into a dictionary with column names as keys
films_data = []
for row in rows:
    film = {column_names[i]: row[i] for i in range(len(column_names))}
    films_data.append(film)

# Export the list of dictionaries to a JSON file
with open('films_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(films_data, json_file, indent=4)

# Close the database connection
conn.close()

print("Database has been successfully exported to films.json.")

Database has been successfully exported to films.json.
